In [ ]:
%reload_ext setup

In [ ]:
from igent.tools import fetch_incentives
import requests, os

def fetch_incentives(language: str = "en", zip_code: str = "55401") -> str:
    """Fetches incentive programs from Rewiring America API.
    
    Args:
    Args:
        language (str): Language code for the response (default: "en")
        zip_code (str): 5-digit US ZIP code (default: "55401" for Minneapolis, MN)
    
    Returns:
        str: JSON string of incentive programs
        
    Raises:
        ValueError: If API key is not configured
    """
    url = "https://api.rewiringamerica.org/api/v1/incentives/programs"
    api_key = os.getenv("REWIRING_AMERICA_API_KEY")
    
    if not api_key:
        raise ValueError("REWIRING_AMERICA_API_KEY environment variable not set")
        
    headers = {"Authorization": f"Bearer {api_key}"}
    params = {
        "language": language,
        "zip": zip_code  # Changed from "state" to "zip" based on error message
    }
    response = requests.get(
        url,
        headers=headers,
        params=params,
        timeout=15
    )
    return response.text

fetch_incentives(language="en")

## Introduction to Agents

In [ ]:
# Cell 1: Imports and Logging Configuration
import logging
from igent.matching_agent import get_agents
from autogen_agentchat.base import TaskResult
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken

# Suppress autogen_core and httpx INFO logs
logging.getLogger("autogen_core").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

# Cell 2: Get the agent
agents = await get_agents(
    state_path="model_config.yaml",
    matcher_prompt="You are a helpful assistant",
    critic_prompt="You are a critic, giving useful fedback. If happy with the result, say 'APPROVE'. If not, give feedback."
)
# display(agents)

# Cell 3: Run the chat with simplified output
message = "Hello, tell a joke?"
async for msg in agents.run_stream(
    task=[TextMessage(content=message, source="user")],
    cancellation_token=CancellationToken()
):
    if isinstance(msg, TextMessage):  # Only show actual text messages between agents
        print(f"{msg.source}: {msg.content}")
    elif isinstance(msg, TaskResult):  # Show final result
        final_message = "Task completed. "
        if msg.stop_reason:
            final_message += f"Stop reason: {msg.stop_reason}"
        print(final_message)

## Demo Example (1 registrants)

In [ ]:
# Cell 1: Imports and Logging Configuration
import logging
from igent.matching_agent import get_agents
from autogen_agentchat.base import TaskResult
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken

logging.getLogger("autogen_core").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

# Cell 2: Read and combine prompt files
def read_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

general_context = read_file('../prompts/general_context.txt')
matching_rules = read_file('../prompts/matching_rules.txt')
distribution_rules = read_file('../prompts/distribution_rules.txt')
matcher_prompt = read_file('../prompts/matcher/matcher_1.txt')
critic_prompt = read_file('../prompts/critic/critic_1.txt')

combined_matcher_prompt = matcher_prompt.format(general_context=general_context, matching_rules=matching_rules, distribution_rules=distribution_rules)
combined_critic_prompt = critic_prompt.format(general_context=general_context, matching_rules=matching_rules, distribution_rules=distribution_rules)


# Cell 3: Get the agent
agents = await get_agents(
    state_path="model_config.yaml",
    matcher_prompt=combined_matcher_prompt,
    critic_prompt=combined_critic_prompt
)

# Cell 3: Run the chat with matching task
message = (
    "Load data from '../data/demo/registrations.json' and '../data/demo/offers.json'. "
    "Match all registrations to suppliers based on ZIP code, product needs (e.g., panels), "
    "and supplier capacity. Ensure proportional distribution across suppliers with overlapping service areas. "
    "Output matches in JSON format as per the matching rules."
)
async for msg in agents.run_stream(
    task=[TextMessage(content=message, source="user")],
    cancellation_token=CancellationToken()
):
    if isinstance(msg, TextMessage):
        print(f"{msg.source}: {msg.content}")
    elif isinstance(msg, TaskResult):
        final_message = "Task completed. "
        if msg.stop_reason:
            final_message += f"Stop reason: {msg.stop_reason}"
        print(final_message)

In [ ]:
# Cell 1: Imports and Logging Configuration
import logging
from igent.matching_agent import get_agents
from autogen_agentchat.base import TaskResult
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken

logging.getLogger("autogen_core").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

# Cell 2: Read and combine prompt files
def read_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

general_context = read_file('../prompts/general_context.txt')
subsidy_matching_rules = read_file('../prompts/subsidy_matching_rules.txt')
pricing_rules = read_file('../prompts/pricing_rules.txt')
subsidy_matcher_prompt = read_file('../prompts/subsidy_matcher/subsidy_matcher_1.txt')
subsidy_critic_prompt = read_file('../prompts/subsidy_critic/subsidy_critic_1.txt')

combined_subsidy_matcher_prompt = subsidy_matcher_prompt.format(general_context=general_context, subsidy_matching_rules=subsidy_matching_rules, pricing_rules=pricing_rules)
combined_subsidy_critic_prompt = subsidy_critic_prompt.format(general_context=general_context, subsidy_matching_rules=subsidy_matching_rules, pricing_rules=pricing_rules)


# Cell 3: Get the agent
agents = await get_agents(
    state_path="model_config.yaml",
    matcher_prompt=combined_subsidy_matcher_prompt,
    critic_prompt=combined_subsidy_critic_prompt
)

# Cell 3: Run the chat with matching task
message = (
    "Load data from 'test.json', '../data/demo/offers.json' and '../data/demo/incentives.json'. "
    "Match all outputs to incentives."
    "Output matches in JSON format as per the matching rules."
)
async for msg in agents.run_stream(
    task=[TextMessage(content=message, source="user")],
    cancellation_token=CancellationToken()
):
    if isinstance(msg, TextMessage):
        print(f"{msg.source}: {msg.content}")
    elif isinstance(msg, TaskResult):
        final_message = "Task completed. "
        if msg.stop_reason:
            final_message += f"Stop reason: {msg.stop_reason}"
        print(final_message)